In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("data/raw/uk-war.csv", names=["date","time","location","lat","lon","kind","description","link"])
df.dropna(axis=0,how='any',subset=['lat','lon'],inplace=True)


In [ ]:
# Variable with the Longitude and Latitude
X=df.loc[:,['kind','lat','lon']]
X = X[X["kind"].str.contains("Ua") == False]
X = X.drop('kind', axis=1)
X.info()

In [ ]:
X=df.loc[:,['kind','lat','lon']]
X = X[X["kind"].str.contains("Ru") == False]
X = X[X["kind"].str.contains("RU") == False]
X = X.drop('kind', axis=1)
X.info()

In [ ]:
# Set up the loop and plot
fig1, axes1 = plt.subplots(2, 3, figsize=(8, 8))
ypts = X.lat.to_numpy()
xpts = X.lon.to_numpy()
alldata = np.vstack((xpts, ypts))
fpcs = []

for ncenters, ax in enumerate(axes1.reshape(-1), 4):
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
        alldata, ncenters, 2, error=0.005, maxiter=1000, init=None)

    # Store fpc values for later
    fpcs.append(fpc)

    # Plot assigned clusters, for each data point in training set
    cluster_membership = np.argmax(u, axis=0)
    for j in range(ncenters):
        ax.plot(xpts[cluster_membership == j],
                ypts[cluster_membership == j], '.', color=colors[j])

    # Mark the center of each fuzzy cluster
    for pt in cntr:
        ax.plot(pt[0], pt[1], 'rs')

    ax.set_title('Centers = {0}; FPC = {1:.3f}'.format(ncenters, fpc))
    ax.axis('off')

fig1.tight_layout()

In [ ]:
ig2, ax2 = plt.subplots()
ax2.plot(np.r_[4:10], fpcs)
ax2.set_xlabel("Number of centers")
ax2.set_ylabel("Fuzzy partition coefficient")

In [ ]:
ypts = X.lat.to_numpy()
xpts = X.lon.to_numpy()
alldata = np.vstack((xpts, ypts))

cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
    alldata, 6, 2, error=0.005, maxiter=1000, init=None)

# Plot assigned clusters, for each data point in training set
cluster_membership = np.argmax(u, axis=0)

for j in range(6):
    plt.plot(xpts[cluster_membership == j],
            ypts[cluster_membership == j], '.',)

# Mark the center of each fuzzy cluster
for pt in cntr:
    plt.plot(pt[0], pt[1], 'rs')

plt.title('Centers = {0}; FPC = {1:.2f}'.format(len(cntr), fpc))

In [ ]:
BBox = ((df.lon.min(),   df.lon.max(),
        df.lat.min(), df.lat.max()))
BBox

In [ ]:
ruh_m = plt.imread('map.jpeg')

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X.lon, X.lat, c=cluster_membership, s=50, zorder=1, alpha= 0.4, marker='*',cmap='viridis')
ax.scatter(pd.Series(cntr[:,0]),pd.Series(cntr[:,1]), c='red', s=200, alpha=0.7)
ax.set_title('Plotting Spatial Data of UA troops on Ukraine Map')
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
ax.imshow(ruh_m, zorder=0, extent = BBox, aspect= 'auto')